Notebook 01 – Análisis Exploratorio del Dataset MSL-150
📌 1. Introducción

Este notebook realiza un análisis exploratorio inicial (EDA) del dataset
MSL-150_Mexican_Sign_Language_Dataset.csv, disponible en Zenodo.

El objetivo principal es:

Verificar la estructura del dataset

Confirmar la presencia de las 150 clases

Identificar número de muestras por clase

Validar coherencia interna en los datos

Explorar distribución, filas, columnas y primeros registros

Este análisis es la base para los notebooks posteriores (preparación de datos, entrenamiento y evaluación de modelos secuenciales).


Notebook 01 – Exploratory Data Analysis (EDA) for MSL-150 Dataset
📌 1. Introduction

This notebook performs an initial exploratory data analysis of the
MSL-150_Mexican_Sign_Language_Dataset.csv, available on Zenodo.

Objectives:

Inspect dataset structure

Confirm the 150 expected classes

Count samples per class

Validate internal consistency

Explore columns, sample rows, and key statistics

This EDA supports the next notebooks on data preparation and sequential model training.

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


print("Librerías cargadas correctamente.")


Librerías cargadas correctamente.


In [2]:
# Ruta base del repositorio
BASE_REPO = "/Users/armandobecerril/PhD/MSL-150"

# -----------------------------
# 1) Datos crudos / CSV maestro
# -----------------------------
# Aquí tienes el CSV grande y el zip con todos los npy tal como vienen de Zenodo
base_dir = f"{BASE_REPO}/data/raw"          # equivale a LSM_DATA
base_dir_csv = base_dir                      # para mantener compatibilidad con código viejo

# Si quieres referirte directamente al CSV:
master_csv_path = f"{base_dir}/MSL-150_Mexican_Sign_Language_Dataset.csv"

# -----------------------------
# 2) CSV estandarizados (si los generas de nuevo)
# -----------------------------
std_dir = f"{BASE_REPO}/data/standardized"   # equivale a LSM_DATA_01_STD_CSV
# (la carpeta se crea cuando corras el pipeline que los genere)

# -----------------------------
# 3) NPY completos y subset demo
# -----------------------------
np_dir_csv     = f"{BASE_REPO}/data/raw_npy"     # equivale a LSM_DATA_03_PREP_NP (todos los .npy)
sample_npy_dir = f"{BASE_REPO}/data/sample_npy"  # subset de 5 palabras para pruebas rápidas

# -----------------------------
# 4) Modelos entrenados (todas las versiones)
# -----------------------------
models_versions = f"{BASE_REPO}/models/trained_models"  # equivale a LSM_DATA_04_MODEL_VERSIONS

# -----------------------------
# 5) Diccionario de términos
# -----------------------------
terms_path = f"{BASE_REPO}/data/dictionary/terms.txt"

# Construcción del diccionario de vocabulario (igual que antes)
frames = 30
samples = 800          # o 200 si quieres que refleje el subset


videos_dict_aug = {}
with open(terms_path, 'r') as file:
    for line in file:
        term = line.strip()
        videos_dict_aug[term] = samples

print(f"Total terms: {len(videos_dict_aug)}")


Total terms: 150


In [3]:
df_head = pd.read_csv(master_csv_path, nrows=5)
print(df_head.columns.tolist())

['VIDEO_SAMPLE', 'CLASSIFICATION', 'FRAME', 'TIMESTAMP', 'RIGHT_WRIST_X', 'RIGHT_WRIST_Y', 'RIGHT_WRIST_Z', 'RIGHT_THUMB_CMC_X', 'RIGHT_THUMB_CMC_Y', 'RIGHT_THUMB_CMC_Z', 'RIGHT_THUMB_MCP_X', 'RIGHT_THUMB_MCP_Y', 'RIGHT_THUMB_MCP_Z', 'RIGHT_THUMB_IP_X', 'RIGHT_THUMB_IP_Y', 'RIGHT_THUMB_IP_Z', 'RIGHT_THUMB_TIP_X', 'RIGHT_THUMB_TIP_Y', 'RIGHT_THUMB_TIP_Z', 'RIGHT_INDEX_FINGER_MCP_X', 'RIGHT_INDEX_FINGER_MCP_Y', 'RIGHT_INDEX_FINGER_MCP_Z', 'RIGHT_INDEX_FINGER_PIP_X', 'RIGHT_INDEX_FINGER_PIP_Y', 'RIGHT_INDEX_FINGER_PIP_Z', 'RIGHT_INDEX_FINGER_DIP_X', 'RIGHT_INDEX_FINGER_DIP_Y', 'RIGHT_INDEX_FINGER_DIP_Z', 'RIGHT_INDEX_FINGER_TIP_X', 'RIGHT_INDEX_FINGER_TIP_Y', 'RIGHT_INDEX_FINGER_TIP_Z', 'RIGHT_MIDDLE_FINGER_MCP_X', 'RIGHT_MIDDLE_FINGER_MCP_Y', 'RIGHT_MIDDLE_FINGER_MCP_Z', 'RIGHT_MIDDLE_FINGER_PIP_X', 'RIGHT_MIDDLE_FINGER_PIP_Y', 'RIGHT_MIDDLE_FINGER_PIP_Z', 'RIGHT_MIDDLE_FINGER_DIP_X', 'RIGHT_MIDDLE_FINGER_DIP_Y', 'RIGHT_MIDDLE_FINGER_DIP_Z', 'RIGHT_MIDDLE_FINGER_TIP_X', 'RIGHT_MIDDLE_FIN

In [7]:
import pandas as pd

label_col = "CLASSIFICATION"

class_counts = {}
chunksize = 50_000

for chunk in pd.read_csv(
    master_csv_path,
    chunksize=chunksize,
    usecols=[label_col],
    dtype={label_col: str}  # 👈 forzamos a string
):
    # Normalizamos en cada chunk
    chunk[label_col] = (
        chunk[label_col]
        .fillna("NaN_VALUE")  # si quieres puedes luego filtrar esto
        .str.strip()
    )

    vc = chunk[label_col].value_counts()
    for label, cnt in vc.items():
        class_counts[label] = class_counts.get(label, 0) + cnt

class_counts_df = (
    pd.DataFrame.from_dict(class_counts, orient="index", columns=["num_samples"])
      .sort_values("num_samples", ascending=False)
)

display(class_counts_df.head(10))
print(f"\nTotal de clases        : {class_counts_df.shape[0]}")
print(f"Mínimo muestras/clase  : {class_counts_df['num_samples'].min()}")
print(f"Máximo muestras/clase  : {class_counts_df['num_samples'].max()}")


,num_samples
hospital,24000
terapia,24000
paciente,24000
enfermero,24000
enfermera,24000
doctor,24000
aborto,24000
virus,24000
jarabe,24000
caminar,24000



Total de clases        : 150
Mínimo muestras/clase  : 8676
Máximo muestras/clase  : 24000


In [6]:
import pandas as pd

# Cargar SOLO CLASSIFICATION, convirtiendo todo a string
labels = pd.read_csv(master_csv_path, usecols=['CLASSIFICATION'], dtype=str)

# Reemplazar NaN por texto explícito
labels['CLASSIFICATION'] = labels['CLASSIFICATION'].fillna("NaN_VALUE")

# Remover espacios alrededor
labels['CLASSIFICATION'] = labels['CLASSIFICATION'].str.strip()

# Obtener lista ordenada
unique_labels = sorted(labels['CLASSIFICATION'].unique())

print("Número de clases:", len(unique_labels))
unique_labels


Número de clases: 150


['1',
 '10',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'abeja',
 'aborto',
 'abril',
 'accidente',
 'agosto',
 'ahora',
 'ambulancia',
 'analisis',
 'ayer',
 'beber',
 'bien',
 'boca',
 'brazo',
 'calentura',
 'caliente',
 'camaron',
 'caminar',
 'cancer',
 'cansado',
 'cintura',
 'cita',
 'cocinar',
 'codo',
 'comer',
 'como',
 'confundido',
 'contagiar',
 'convulciones',
 'coronavirus',
 'correr',
 'cuantos',
 'cuello',
 'debil',
 'descansar',
 'diario',
 'diarrea',
 'diciembre',
 'doctor',
 'dolor',
 'domingo',
 'dormir',
 'duda',
 'duro',
 'embarazo',
 'emergencia',
 'enero',
 'enfermera',
 'enfermero',
 'enfermo',
 'espalda',
 'esposa',
 'esposo',
 'estresado',
 'estudiar',
 'farmacia',
 'febrero',
 'fractura',
 'frio',
 'garganta',
 'gases',
 'gato',
 'gripa',
 'hija',
 'hijo',
 'hombro',
 'hospital',
 'hoy',
 'huesos',
 'infarto',
 'infeccion',
 'inflamacion',
 'interpretar',
 'inyeccion',
 'ir',
 'jarabe',
 'jueves',
 'julio',
 'junio',
 'lento',
 'lesion',
 'lunes',
 'm